In [23]:
import yfinance as yf
import pandas as pd

df = yf.download('BTC-USD', start='2025-01-01', end='2025-09-22', interval='1wk')


/tmp/ipykernel_599072/2684615471.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('BTC-USD', start='2025-01-01', end='2025-09-22', interval='1wk')
[*********************100%***********************]  1 of 1 completed


In [24]:
data = df.copy()

In [25]:
print(df.head())
df["Date"] = pd.to_datetime(df.index, utc=True)
print(df.head())

Price               Close           High           Low           Open  \
Ticker            BTC-USD        BTC-USD       BTC-USD        BTC-USD   
Date                                                                    
2025-01-01   96922.703125  102712.484375  92788.125000   93425.101562   
2025-01-08   96534.046875   97352.664062  89260.101562   96924.164062   
2025-01-15  106146.265625  109114.882812  96501.640625   96534.046875   
2025-01-22  101332.476562  107098.546875  97795.937500  106136.382812   
2025-01-29   97871.820312  106418.765625  91242.890625  101317.523438   

Price             Volume  
Ticker           BTC-USD  
Date                      
2025-01-01  259517878648  
2025-01-08  355206513683  
2025-01-15  526047818939  
2025-01-22  392990612048  
2025-01-29  414333348359  
Price               Close           High           Low           Open  \
Ticker            BTC-USD        BTC-USD       BTC-USD        BTC-USD   
Date                                                 

In [26]:
df["Date"] = pd.to_datetime(df["Date"]).astype(int) / 10**9


In [27]:

df.reset_index(drop=True, inplace=True)


In [28]:
df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
df.to_csv("dataset/cryptex/candlesticks-W-full.csv", index=False)

In [34]:
df = pd.read_csv("dataset/cryptex/candlesticks-W-2025.csv")


In [35]:
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
df.columns = ["timestamp", "open", "high", "low", "close", "volume"]

df.to_csv("dataset/cryptex/candlesticks-W-2025.csv", index=False)


TypeError: cannot convert the series to <class 'int'>

In [2]:
import numpy as np
def convert_to_returns(data, log_returns=False):
    """
    Convert data to returns.

    args:
        data: pandas DataFrame with "close" and "volume" columns
        log_returns: bool, if True, the data is converted to log returns
    returns:
        pandas DataFrame with "returns" and "volume" columns
    """
    data = pd.DataFrame({"close": data["close"], "volume": data["volume"], "timestamp": data["timestamp"]})
    if log_returns:
        data["returns"] = np.log(data["close"] / data["close"].shift(1))
    else:
        data["returns"] = data["close"] / data["close"].shift(1) - 1
    data = data.drop("close", axis=1).dropna().reset_index(drop=True)

    data = data[["timestamp", "returns", "volume"]]

    return data


In [38]:
df = pd.read_csv("dataset/cryptex/candlesticks-W-2014-2024.csv")


In [40]:
df = convert_to_returns(df)

KeyError: 'close'

In [42]:
df.to_csv("dataset/cryptex/returns-2014-2024.csv", index=False)

In [3]:
import pandas as pd
df = pd.read_csv("dataset/cryptex/candlesticks-W-2025.csv")

df = convert_to_returns(df)
df.to_csv("dataset/cryptex/returns-2025.csv", index=False)




In [8]:
def convert_back_to_candlesticks(candlesticks, predicted_returns):
    """
    Convert returns data back to candlesticks.

    args:
        data: pandas DataFrame with "returns" and "volume" columns
        """
    
    # Make a copy of the candlesticks data
    result = candlesticks.copy()
    
    # Get the last known close price before predictions start
    last_close = result.loc[result.index[predicted_returns['returns_predicted_1'].first_valid_index()-1], 'close']
    
    # Calculate predicted close prices from returns
    for i in range(1, 3):  # For returns_predicted_1 and returns_predicted_2
        col = f'returns_predicted_{i}'
        if col in predicted_returns.columns:
            # Calculate cumulative returns 
            pred_close = last_close * (1 + predicted_returns[col])
            # Rename column
            result[f'close_predicted_{i}'] = pred_close
    
    return result

In [9]:
candles = pd.read_csv("dataset/cryptex/candlesticks-W-2025.csv")
pred_returns = pd.read_csv("backtesting/data/inference_returns_w.csv")






In [10]:
df = convert_back_to_candlesticks(candles, pred_returns)
df.head()

,timestamp,open,high,low,close,volume,close_predicted_1,close_predicted_2
0,1.735690e+09,96922.703125,102712.484375,92788.125000,93425.101562,259517878648,NaN,NaN
1,1.736294e+09,96534.046875,97352.664062,89260.101562,96924.164062,355206513683,NaN,NaN
2,1.736899e+09,106146.265625,109114.882812,96501.640625,96534.046875,526047818939,NaN,NaN
3,1.737504e+09,101332.476562,107098.546875,97795.937500,106136.382812,392990612048,NaN,NaN
4,1.738109e+09,97871.820312,106418.765625,91242.890625,101317.523438,414333348359,NaN,NaN


In [15]:
df.drop(columns=["close_predicted_2"]).to_csv("backtesting/data/returns_backtest.csv", index=False)

In [12]:
df2 = pd.read_csv("backtesting/data/iar_LLAMA3.1_L12_daily_S_seq168_pred1_p6_s4_v100.csv")

,timestamp,open,high,low,close,volume,close_predicted_1
0,2021-06-25,34659.10547,35487.24609,31350.88477,31637.77930,4.023090e+10,NaN
1,2021-06-26,31594.66406,32637.58789,30184.50195,32186.27734,3.858539e+10,NaN
2,2021-06-27,32287.52344,34656.12891,32071.75781,34649.64453,3.551164e+10,NaN
3,2021-06-28,34679.12109,35219.89063,33902.07422,34434.33594,3.389252e+10,NaN
4,2021-06-29,34475.55859,36542.10938,34252.48438,35867.77734,3.790146e+10,NaN
...,...,...,...,...,...,...,...
1457,2025-06-21,103315.07810,104015.78130,100973.06250,102257.40630,3.836056e+10,97599.953125
1458,2025-06-22,102212.03130,103351.63280,98286.20313,100987.14060,6.553700e+10,97574.070312
1459,2025-06-23,100987.47660,106116.85940,99705.75000,105577.77340,6.523776e+10,97715.023438
1460,2025-06-24,105571.51560,106316.82810,104740.24220,106045.63280,4.882299e+10,97714.070312
